In [ ]:
import matplotlib.pyplot as plt
import polars as pl
import torch
import matplotlib
import cartopy.crs as ccrs
import numpy as np
from pathlib import Path


plt.style.use(Path("../meta/colorblind_friendly.mplstyle"))
# fonts bigger than usual because of how i built the axes
pgf_with_latex = {
    "font.family": "serif",
    "font.size": 12,
    "axes.labelsize": 12,
    "axes.titlesize": 12,
    "legend.fontsize": 12,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "figure.titlesize": 12,
    "axes.spines.top": False,
    "axes.spines.right": False,
}
matplotlib.rcParams.update(pgf_with_latex)

plt.style.use(Path("../meta/colorblind_friendly.mplstyle"))
matplotlib.rcParams.update({"font.family": "serif", "font.size": 8})


# station locations
stationlist_all = pl.read_csv("../meta/stations_all.csv")
stations_all = torch.tensor(
    np.vstack((stationlist_all["X"].to_numpy(), stationlist_all["Y"].to_numpy())).T
)

# receiver stations
stationlist_rcv = pl.read_csv("../meta/stations_receivers.csv")
names_receivers = stationlist_rcv["station"]
stations_receivers = torch.tensor(
    np.vstack((stationlist_rcv["X"].to_numpy(), stationlist_rcv["Y"].to_numpy())).T
)

# auxiliary stations
stationlist_aux = pl.read_csv("../meta/stations_auxiliary.csv")
stations_auxiliary = torch.tensor(
    np.vstack((stationlist_aux["X"].to_numpy(), stationlist_aux["Y"].to_numpy())).T
)

master_idx = names_receivers.to_numpy().tolist().index("OMV.GDT")

_cm = 1 / 2.54
fig, ax = plt.subplots(
    figsize=(10 * _cm, 2 / 3 * 10 * _cm),
)
ax.scatter(*stations_all.T, s=2, lw=0, c="#DDD")
ax.scatter(
    *stations_receivers.T,
    s=6,
    lw=0,
    marker="v",
    c="#3F90DA",
    label="Receiver station",
)
ax.scatter(
    *stations_receivers[master_idx].T,
    s=50,
    lw=0.5,
    ec="k",
    marker="v",
    c="#FFA90E",
    label="Master station",
)
ax.scatter(
    *stations_auxiliary.T,
    s=6,
    lw=0,
    marker="v",
    c="#832DB6",
    label="Auxiliary station",
)

xticks = [-10, -5, 0, 5, 10]
yticks = [-5, 0, 5]
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_xticklabels(xticks)
ax.set_yticklabels(yticks)
ax.set_xlabel("Distance [km]", labelpad=0)
ax.set_ylabel("Distance [km]", labelpad=-4)

# get the current x and y limits
xlim = ax.get_xlim()
ylim = ax.get_ylim()

ax.add_patch(
    matplotlib.patches.Rectangle(
        (-4.5, -4.5),
        9,
        9,
        linewidth=1.5,
        ls="--",
        edgecolor="k",
        facecolor="none",
        zorder=10,
    )
)

ax.annotate(
    "Figs. 3 – 8",
    xy=(-5, 4.5),
    xytext=(-5, 5.5),
    fontsize=10,
    ha="right",
    va="bottom",
    arrowprops=dict(arrowstyle="->", color="k", connectionstyle="arc3,rad=0.1"),
)

ax.set_title("b)", loc="left", pad=5)

# add an ax in bottom right corner
ax2 = fig.add_axes(
    [-0.1, 0.35, 0.25, 0.25],
    projection=ccrs.UTM(33),
)
# approximate array location
ref_lon, ref_lat = 16.7, 48.4
ax2.set_extent(
    [
        ref_lon - 1,
        ref_lon + 1,
        ref_lat - 0.5,
        ref_lat + 0.5,
    ],
    crs=ccrs.PlateCarree(),
)

# mark view of ax in ax2 with a box
ax2_x0, ax2_x1, ax2_y1, ax2_y2 = ax2.get_extent()
ax2.set_facecolor("none")
ax2.axis("off")

# another ax with even wider region that highlights the area in ax2
ax3 = fig.add_axes([-0.125, 0.63, 0.25, 0.25], projection=ccrs.Mercator())
ax3.set_extent(
    [
        ref_lon - 9 - 4,
        ref_lon + 9 - 4,
        ref_lat - 6,
        ref_lat + 8,
    ],
    crs=ccrs.PlateCarree(),
)
ax3.coastlines(lw=0.5)
ax3.add_feature(ccrs.cartopy.feature.BORDERS, lw=0.5, color="#CCC")
ax3.add_feature(ccrs.cartopy.feature.OCEAN, lw=0.5, color="#b2cce1")
ax3.add_patch(
    matplotlib.patches.Rectangle(
        (ax2_x0, ax2_y1),
        ax2_x1 - ax2_x0,
        ax2_y2 - ax2_y1,
        linewidth=1,
        edgecolor="k",
        facecolor="none",
        transform=ccrs.UTM(33),
        zorder=10,
    )
)

ax3.set_title("a)", loc="left", pad=5)

ax4 = ax3.inset_axes(
    [0.0, -1.0, 1, 1],  # [left, bottom, width, height]
    xlim=(ax2_x0, ax2_x1),
    ylim=(ax2_y1, ax2_y2),
    projection=ccrs.UTM(33),
)
ax4.coastlines(lw=0.5)
ax4.add_feature(ccrs.cartopy.feature.BORDERS, lw=0.5, color="#CCC")

# approximate box of cartesian grid
ax4.add_patch(
    matplotlib.patches.Rectangle(
        (ref_lon - 0.11, ref_lat - 0.065),
        0.22,
        0.13,
        linewidth=1,
        edgecolor="k",
        facecolor="none",
        transform=ccrs.PlateCarree(),
        zorder=10,
    )
)

# indicate zoom dashed lines to towards main ax
ax4.plot(
    [0.5, 1.7],
    [0.4, -1.95],
    transform=ax4.transAxes,
    clip_on=False,
    c="#666",
    dashes=[8, 6],
    lw=0.5,
    zorder=10,
)
ax4.plot(
    [0.5, 1.7],
    [0.6, 2.8],
    transform=ax4.transAxes,
    clip_on=False,
    c="#666",
    dashes=[8, 6],
    lw=0.5,
    zorder=10,
)

# custom legend for these three station types to ensure they're legible
handles = [
    plt.Line2D(
        [0],
        [0],
        marker="v",
        color="w",
        label="Master station",
        markerfacecolor="#FFA90E",
        markersize=9,
    ),
    plt.Line2D(
        [0],
        [0],
        marker="v",
        color="w",
        label="Receiver station",
        markerfacecolor="#3F90DA",
        markersize=7,
    ),
    plt.Line2D(
        [0],
        [0],
        marker="v",
        color="w",
        label="Auxiliary station",
        markerfacecolor="#832DB6",
        markersize=7,
    ),
    plt.Line2D(
        [0],
        [0],
        marker="*",
        color="w",
        label="Seismic source",
        markerfacecolor="#94A4A2",
        markersize=10,
    ),
]

ax4.set_xlabel("")
ax4.set_ylabel("")

# another axis for synthetic setup
x0, y0, w, h = ax.get_position().bounds
ax5 = fig.add_axes(
    [x0 + w, y0, w, h],
)

ax5.scatter(
    *stations_receivers.T,
    s=0.5,
    lw=0,
    c="#3F90DA",
)
ax5.scatter(
    *stations_receivers[master_idx].T,
    s=20,
    marker="v",
    ec="k",
    lw=0.5,
    c="#FFA90E",
)
ax5.scatter(
    *stations_auxiliary.T,
    s=0.5,
    lw=0,
    c="#832DB6",
)

# sources in a circle surrounding the array
n_boundary_sources = 100
boundary_source_angles = np.linspace(
    0, 2 * np.pi - 2 * np.pi / n_boundary_sources, n_boundary_sources
)
boundary_source_radius = 50
boundary_sources = torch.tensor(
    np.stack(
        [
            boundary_source_radius * np.cos(boundary_source_angles),
            boundary_source_radius * np.sin(boundary_source_angles),
        ],
        axis=1,
    )
)

# additional cluster of sources in the northwest of the boundary sources
n_cluster_sources = 25
cluster_spread = 25
x_center, y_center = (
    boundary_source_radius * np.cos(0.8 * np.pi),
    boundary_source_radius * np.sin(0.8 * np.pi),
)

torch.manual_seed(42)
cluster_sources = (
    torch.rand(n_cluster_sources, 2) * cluster_spread - cluster_spread / 2
) + torch.tensor([x_center, y_center])

sources = torch.cat([boundary_sources, cluster_sources], dim=0).float()

ax5.scatter(
    *sources.T,
    s=25,
    lw=0,
    marker="*",
    c="#94A4A2",
)

ax5.set_aspect("equal")
ax5.set_xlim(-60, 60)
ax5.set_ylim(-60, 60)
xticks = [-50, 0, 50]
yticks = [-50, 0, 50]
ax5.set_xticks(xticks)
ax5.set_yticks(yticks)
ax5.set_xticklabels(xticks)
ax5.set_yticklabels(yticks)
ax5.set_xlabel("Distance [km]", labelpad=0)
ax5.set_title("c)", loc="left", pad=5)

# move ax to the right a bit
x0, y0, w, h = ax.get_position().bounds
ax.set_position([x0 + 0.05, y0, w, h])

# add one more ax to the right, same size as c)
x0, y0, w, h = ax5.get_position().bounds
ax6 = fig.add_axes(
    [x0 + w + 0.1, y0, w, h],
)

# Far field stuff
# geometry
grid_spacing = 0.5
x_coords = torch.arange(-35, 35 + grid_spacing, grid_spacing).float()
y_coords = torch.arange(-35, 35 + grid_spacing, grid_spacing).float()
array_stations = grid_points = torch.cartesian_prod(x_coords, y_coords)

# auxiliary stations (smaller circle of them)
auxiliary_radius = 75
n_aux = 180
auxiliary_angles = np.linspace(0, 2 * np.pi, n_aux, endpoint=False)
aux_stations = torch.tensor(
    np.stack(
        [
            auxiliary_radius * np.cos(auxiliary_angles),
            auxiliary_radius * np.sin(auxiliary_angles),
        ],
        axis=1,
    )
).float()

# define source locations
# sources in a circle surrounding the array
n_boundary_sources = 180
boundary_source_angles = np.linspace(
    0, 2 * np.pi - 2 * np.pi / n_boundary_sources, n_boundary_sources
)
boundary_source_radius = 100
boundary_sources = torch.tensor(
    np.stack(
        [
            boundary_source_radius * np.cos(boundary_source_angles),
            boundary_source_radius * np.sin(boundary_source_angles),
        ],
        axis=1,
    )
)

# additional cluster of sources in the northwest of the boundary sources
n_cluster_sources = 6
cluster_spread = 0
x_center, y_center = (
    boundary_source_radius * np.cos(0.8 * np.pi),
    boundary_source_radius * np.sin(0.8 * np.pi),
)
cluster_sources = (
    torch.rand(n_cluster_sources, 2) * cluster_spread - cluster_spread / 2
) + torch.tensor([x_center, y_center])

sources = torch.cat([boundary_sources, cluster_sources], dim=0).float()

ax6.scatter(
    *array_stations.T,
    s=0.5,
    lw=0,
    c="#3F90DA",
    label="Receiver station",
)
ax6.scatter(
    *stations_receivers[master_idx].T,
    s=20,
    marker="v",
    ec="k",
    lw=0.5,
    label="Master station",
    c="#FFA90E",
)
ax6.scatter(
    *aux_stations.T,
    s=2,
    lw=0,
    label="Auxiliary station",
    c="#832DB6",
)
ax6.scatter(
    *sources.T,
    s=25,
    lw=0,
    marker="*",
    c="#94A4A2",
    label="Source",
)
ax6.scatter(
    *cluster_sources.T,
    s=250,
    lw=0,
    marker="*",
    c="#94A4A2",
)

ax6.set_aspect("equal")
ax6.set_xlabel("Distance [km]", labelpad=0)
xticks = [-100, 0, 100]
yticks = [-100, 0, 100]
ax6.set_xticks(xticks)
ax6.set_yticks(yticks)
ax6.set_xticklabels(xticks)
ax6.set_yticklabels(yticks)
ax6.set_title("d)", loc="left", pad=5)

# add a box -4.5 to 4.5 km around the master station (but in cartesian coordinates)
ax6.add_patch(
    matplotlib.patches.Rectangle(
        (-35, -35),
        70,
        70,
        linewidth=1.5,
        ls="--",
        edgecolor="k",
        facecolor="none",
        zorder=10,
    )
)

ax6.annotate(
    "Fig. 9",
    xy=(-35, 35),
    xytext=(-55, 90),
    fontsize=10,
    ha="right",
    va="bottom",
    arrowprops=dict(arrowstyle="->", color="k", connectionstyle="arc3,rad=0.1"),
)


ax.legend(
    handles=handles[:-1],
    loc="upper right",
    fontsize=7,
    bbox_to_anchor=(1, 1.1),
    frameon=True,
)

ax5.legend(
    handles=[handles[-1]],
    loc="upper right",
    fontsize=7,
    bbox_to_anchor=(1, 1.1),
    frameon=True,
)

fig.savefig("../figures/figure2.png", bbox_inches="tight", dpi=300)